In [4]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = "昏峰"    #晨峰或昏峰

inputLocation = f"{year}_現況_{rush_hour}/"

################################

import pandas as pd

df= pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]
if rush_hour == '晨峰':
    real_flow = df["晨峰實際流量"]
elif rush_hour == '昏峰':
    real_flow = df["昏峰實際流量"]
print(f'——————————————————————校估{year}年{rush_hour}指派流量——————————————————————')

real_flow = real_flow.set_index("group")


ue_results = pd.read_csv(inputLocation + f"{year}_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)

#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet1["link_name1"] = sheet1.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row

#reserve the link,AM,UE_flow
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['UE_flow_1'] += row['UE_flow_1']
    groups[group]['UE_flow_2'] += row['UE_flow_2']

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
# {real_flow.loc[group, 'name']}
for group, flows in groups.items():
    print(f"""【{group}】 {real_flow.loc[group, 'name']}
    UE flow 往南/東 = {round(flows['UE_flow_1'])}， diff = {(flows['UE_flow_1'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%， CUBE diff = {(flows['UE_flow_1'] - real_flow.loc[group, 'CUBE南/東']) / real_flow.loc[group, 'CUBE南/東']* 100:.2f}%
    UE flow 往北/西 = {round(flows['UE_flow_2'])}， diff = {(flows['UE_flow_2'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%， CUBE diff = {(flows['UE_flow_2'] - real_flow.loc[group, 'CUBE北/西']) / real_flow.loc[group, 'CUBE北/西']* 100:.2f}%
    """)

——————————————————————校估104年昏峰指派流量——————————————————————
【SL1】 基隆河
    UE flow 往南/東 = 41748， diff = 11.88%， CUBE diff = 17.04%
    UE flow 往北/西 = 40169， diff = 23.82%， CUBE diff = 26.71%
    
【SL2】 台鐵(市民大道)
    UE flow 往南/東 = 37742， diff = 13.38%， CUBE diff = 5.24%
    UE flow 往北/西 = 30681， diff = -3.49%， CUBE diff = -7.22%
    
【SL3】 國1-市中心
    UE flow 往南/東 = 4148， diff = -21.69%， CUBE diff = -21.74%
    UE flow 往北/西 = 5408， diff = -1.48%， CUBE diff = 22.59%
    
【SL5】 臺北縣市東側運輸走廊
    UE flow 往南/東 = 14508， diff = -5.18%， CUBE diff = 15.85%
    UE flow 往北/西 = 11668， diff = 3.82%， CUBE diff = 27.18%
    
【SL6】 北市北側
    UE flow 往南/東 = 2935， diff = -28.87%， CUBE diff = -24.38%
    UE flow 往北/西 = 6093， diff = 10.33%， CUBE diff = 1.17%
    
【SL7】 臺北縣市西側運輸走廊(淡水河流域)
    UE flow 往南/東 = 28196， diff = 11.70%， CUBE diff = 1.33%
    UE flow 往北/西 = 36920， diff = 21.98%， CUBE diff = 11.06%
    
【SL8】 臺北縣市南側運輸走廊(新店溪&景美溪流域)
    UE flow 往南/東 = 35665， diff = 10.82%， CUBE diff = -0.79%
    UE flow 往北/西 = 

In [5]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table

,group,link,UE_flow_1,UE_flow_2
4,SL1,中山橋,1227.937721,416.382782
16,SL1,南湖大橋,1794.038535,2627.094465
5,SL1,大直橋,2467.151816,2355.716935
15,SL1,成功橋,1960.946193,1641.491366
14,SL1,成美橋,1718.664390,1224.443164
2,SL1,承德橋,738.357428,79.159321
3,SL1,承德橋(機車道),1806.590399,2093.650951
8,SL1,民權大橋,1378.215496,1157.079175
9,SL1,民權大橋(機車道),1832.133839,1887.397589
0,SL1,洲美快速道路,1978.403013,4358.984185
